In [2]:
import sys
sys.path.append('/workspace/Documents')
import os
import numpy as np
import pandas as pd
import nibabel as nb
import shutil
import Diffusion_for_CT_motion.utils.functions_collection as ff
import matplotlib.pyplot as plt

data_path = '/mnt/camca_NAS/diffusion_ct_motion/models/'
data_path_fixed_CT = '/mnt/camca_NAS/Portable_CT_data/nii_imgs_202404/motion'
save_path = '/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data'


/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Step 1: run main_sample_EDM_for_realdata.py

# Step 2: Prepare data for reader study. saved in diffusion_ct_motion/examples/real_data

In [3]:
patient_list = pd.read_excel('/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/list/NEW_CT_concise_collected_portable_motion_w_fixed_CT_info_Diffusion_results.xlsx',  dtype={'Patient_ID': str, 'Patient_subID': str, 'Patient_ID_fixed':str, 'Patient_subID_fixed':str})
folder = os.path.join(data_path ,'portable_EDM_patch_3Dmotion_hist_v1/pred_images_portable_real_resample_avg')

In [229]:
random_index = 1181
patient = patient_list.loc[patient_list['index'] == random_index]
patient_id = patient['Patient_ID'].values[0]
patient_subid = patient['Patient_subID'].values[0]
print(patient_id , patient_subid)

final_img = nb.load(os.path.join(folder, patient_id, patient_subid,'random_1/epoch49','pred-epoch-49.nii.gz')).get_fdata()
               

00009078 0000156744
(252, 252, 55)


In [230]:
# save corrected and original portable CT
affine = nb.load(os.path.join(folder, patient_id, patient_subid,'random_1/epoch49','pred-epoch-49.nii.gz')).affine
final_img = nb.Nifti1Image(final_img, affine)
ff.make_folder([os.path.join(save_path, patient_id), os.path.join(save_path, patient_id, patient_subid)])
nb.save(final_img, os.path.join(save_path, patient_id, patient_subid, 'corrected.nii.gz'))
shutil.copyfile(os.path.join(folder, patient_id, patient_subid,'random_1/epoch49','motion.nii.gz'), os.path.join(save_path, patient_id, patient_subid , 'original.nii.gz'))

'/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/00009078/0000156744/original.nii.gz'

In [231]:
# save comparing fixed CT
print(patient['Patient_ID_fixed'].values[0], patient['Patient_subID_fixed'].values[0])
fixed_ct_path = os.path.join(data_path_fixed_CT, patient['Patient_ID_fixed'].values[0], patient['Patient_subID_fixed'].values[0], 'fixed/img_resample_avg.nii.gz')
fixed_ct_data = nb.load(fixed_ct_path).get_fdata()
nb.save(nb.Nifti1Image(fixed_ct_data, affine), os.path.join(save_path, patient_id, patient_subid, 'fixed_CT.nii.gz'))


00009078 0000455016


In [ ]:
# assert we finish all the cases
patient_list = pd.read_excel('/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/list/NEW_CT_concise_collected_portable_motion_w_fixed_CT_info_Diffusion_results.xlsx',  dtype={'Patient_ID': str, 'Patient_subID': str, 'Patient_ID_fixed':str, 'Patient_subID_fixed':str})
patient_list = patient_list[patient_list['use_portable']=='yes']
print(patient_list.shape)

for i in range(0, patient_list.shape[0]):
    row = patient_list.iloc[i]
    patient_id = row['Patient_ID']
    patient_subid = row['Patient_subID']

    folder = os.path.join(save_path, patient_id, patient_subid)

    if not os.path.exists(folder):
        print(patient_id, patient_subid)

# Step 3: using nii_to_dcm.m in matlab to prepare the corresponding DICOM files

# Step 4: make blind study

In [5]:
patient_sheet = pd.read_excel('/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/list/NEW_CT_concise_collected_portable_motion_w_fixed_CT_info_Diffusion_results.xlsx',  dtype={'Patient_ID': str, 'Patient_subID': str, 'Patient_ID_fixed':str, 'Patient_subID_fixed':str})
patient_sheet = patient_sheet[patient_sheet['use_portable']=='yes']
print(patient_sheet.shape)


random_save_path = os.path.join(os.path.dirname(save_path), 'randomized')
pilot_save_path = os.path.join(random_save_path, 'pilot')
full_study_save_path = os.path.join(random_save_path, 'full_study')
ff.make_folder([random_save_path, pilot_save_path, full_study_save_path])

(72, 97)


In [6]:
# for pilot study, randomly pick 5 cases, prepare all of original, corrected and fixed CT
pilot_study_case_index = [3,40,55,63,66]
for i in range(0, len(pilot_study_case_index)):
    index = pilot_study_case_index[i]
    row = patient_sheet[patient_sheet['random_index']==index]
    
    patient_id = row['Patient_ID'].values[0]
    patient_subid = row['Patient_subID'].values[0]
    print(patient_id, patient_subid)

    data_folder = os.path.join(save_path, patient_id, patient_subid, 'dicoms')

    save_folder = os.path.join(pilot_save_path, 'random_' + str(index))

    if not os.path.exists(save_folder):
        shutil.copytree(data_folder, save_folder)


00214840 0000455401
00046297 0000455729
00102121 0000156734
00214780 0000455257
00214743 0000455056


In [7]:
import random
# for ful study, prepare three groups with washout
pilot_study_case_index = [3, 40, 55, 63, 66]
patient_sheet_full_study = patient_sheet[~patient_sheet['random_index'].isin(pilot_study_case_index)]

# get a random list for three groups
random_list = []
for i in range(0, patient_sheet_full_study.shape[0]):
   
    a = ['original', 'corrected', 'fixedCT']
    random.shuffle(a)

    random_list.append(a)

wash_group1 = []
wash_group2 = []
wash_group3 = []
for i in range(0,len(random_list)):
    wash_group1.append(random_list[i][0])
    wash_group2.append(random_list[i][1])
    wash_group3.append(random_list[i][2])

patient_sheet_full_study['wash_group1'] = wash_group1
patient_sheet_full_study['wash_group2'] = wash_group2
patient_sheet_full_study['wash_group3'] = wash_group3

# save patient_sheet
# patient_sheet_full_study.to_excel(os.path.join('/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/list', 'NEW_CT_concise_collected_portable_motion_w_fixed_CT_info_Diffusion_results_blinded_reference.xlsx'), index = False)

/tmp/ipykernel_2788683/477130709.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_sheet_full_study['wash_group1'] = wash_group1
/tmp/ipykernel_2788683/477130709.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_sheet_full_study['wash_group2'] = wash_group2
/tmp/ipykernel_2788683/477130709.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [8]:
patient_sheet_full_study = pd.read_excel('/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/list/NEW_CT_concise_collected_portable_motion_w_fixed_CT_info_Diffusion_results_blinded_reference.xlsx',  dtype={'Patient_ID': str, 'Patient_subID': str, 'Patient_ID_fixed':str, 'Patient_subID_fixed':str})
print(patient_sheet_full_study.shape)

for washT in range(1,4):
    ff.make_folder([os.path.join(full_study_save_path, 'washout_' + str(washT))])
    for i in range(0, patient_sheet_full_study.shape[0]):
        row = patient_sheet_full_study.iloc[i]
        patient_id = row['Patient_ID']
        patient_subid = row['Patient_subID']
        random_index = row['random_index']
        wash_group = row['wash_group' + str(washT)]
        print(patient_id, patient_subid, random_index)

        data_folder = os.path.join(save_path, patient_id, patient_subid, 'dicoms_no_description', wash_group)

        save_folder = os.path.join(full_study_save_path, 'washout_' + str(washT), 'random_' + str(random_index))

        print(data_folder, save_folder)

        if not os.path.exists(save_folder):
            shutil.copytree(data_folder, save_folder)

(67, 100)
00213349 0000446627 1
/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/00213349/0000446627/dicoms_no_description/fixedCT /mnt/camca_NAS/diffusion_ct_motion/examples/real_data/randomized/full_study/washout_1/random_1
00154165 0000446630 2
/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/00154165/0000446630/dicoms_no_description/corrected /mnt/camca_NAS/diffusion_ct_motion/examples/real_data/randomized/full_study/washout_1/random_2
00072766 0000455301 4
/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/00072766/0000455301/dicoms_no_description/fixedCT /mnt/camca_NAS/diffusion_ct_motion/examples/real_data/randomized/full_study/washout_1/random_4
00102372 0000455339 5
/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/00102372/0000455339/dicoms_no_description/fixedCT /mnt/camca_NAS/diffusion_ct_motion/examples/real_data/randomized/full_study/washout_1/random_5
00214767 0000455182 6
/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/002

In [42]:
a = ff.find_all_target_files(['*/*/dicoms_no_description/fixed'], '/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data')
for b in a:
    new_folder_name = os.path.dirname(b) + '/fixedCT'
    os.rename(b, new_folder_name)
    print(new_folder_name)


/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/00009078/0000156737/dicoms_no_description/fixedCT
/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/00009078/0000156744/dicoms_no_description/fixedCT
/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/00009962/0000455197/dicoms_no_description/fixedCT
/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/00010469/0000029413/dicoms_no_description/fixedCT
/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/00010804/0000455800/dicoms_no_description/fixedCT
/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/00011588/0000455843/dicoms_no_description/fixedCT
/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/00011616/0000455792/dicoms_no_description/fixedCT
/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/00011638/0000455781/dicoms_no_description/fixedCT
/mnt/camca_NAS/diffusion_ct_motion/examples/real_data/data/00019712/0000455758/dicoms_no_description/fixedCT
/mnt/camca_NAS/diff